Copyright 2016 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

--------------------------------------

This notebook is similar in functionality to [this python script](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/mnist_tflearn.py), and is used with [this README](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/02_README_mnist_tflearn.md).  It shows how to use TensorFlow's high-level apis, in `contrib.tflearn`, to easily build a classifier with multiple hidden layers.

First, do some imports and set some variables:

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import time

import numpy
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

DATA_DIR = "/tmp/MNIST_data"

# read in data, downloading first as necessary
DATA_SETS = input_data.read_data_sets(DATA_DIR)

# comment out for less info during the training runs.
tf.logging.set_verbosity(tf.logging.INFO)

# define a utility function for generating a new directory in which to save 
# model information, so multiple training runs don't stomp on each other.
def getNewPath(base="/tmp/tfmodels/mnist_tflearn"):
    logpath = os.path.join(base, str(int(time.time())))
    print("Logging to {}".format(logpath))
    return logpath


We'll first define a function that adds a LinearClassifier and runs its `fit()` method, which will train the model. Note that we didn't need to explicitly define a model graph or a training loop ourselves.  

Once we've trained the model, we run the `evaluate()` method, which uses the trained model. To do this, it loads the most recent checkpointed model info available.  The model checkpoint(s) will be generated during the training process.

In [ ]:
def define_and_run_linear_classifier(num_steps, logdir):
    """Run a linear classifier."""
    global DATA_SETS
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(
        DATA_SETS.train.images)
    classifier = tf.contrib.learn.LinearClassifier(
                feature_columns=feature_columns, 
                n_classes=10,
               model_dir=logdir
                )
    classifier.fit(DATA_SETS.train.images,
                   DATA_SETS.train.labels.astype(numpy.int64),
                   batch_size=100, steps=num_steps
                  )
    print("Finished training.")
    # Evaluate accuracy.
    accuracy_score = classifier.evaluate(
        DATA_SETS.test.images,
        DATA_SETS.test.labels.astype(numpy.int64))['accuracy']
    print('Linear Classifier Accuracy: {0:f}'.format(accuracy_score))

Next, add a function that defines a `DNNClassifier`, and runs its `fit()` method, which will train the model. Again note that we didn't need to explicitly define a model graph or a training loop ourselves.  

Then after we've trained the model, we run the classifier's `evaluate()` method, which uses the trained model. 

In [ ]:
def define_and_run_dnn_classifier(num_steps, logdir, lr=.1):
    """Run a DNN classifier."""
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(DATA_SETS.train.images)
    classifier = tf.contrib.learn.DNNClassifier(
        feature_columns=feature_columns, n_classes=10,
        hidden_units=[128, 32],
        # After you've done a training run with optimizer learning rate 0.1,
        # change it to 0.5 and run the training again.  Use TensorBoard to take
        # a look at the difference.  You can see both runs by pointing it to the
        # parent model directory, which by default is:
        #   tensorboard --logdir=/tmp/tfmodels/mnist_tflearn
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
        )
    classifier.fit(DATA_SETS.train.images,
                   DATA_SETS.train.labels.astype(numpy.int64),
                   batch_size=100, max_steps=num_steps)
    print("Finished running the training via the fit() method")
    
    accuracy_score = classifier.evaluate(
        DATA_SETS.test.images,
        DATA_SETS.test.labels.astype(numpy.int64))['accuracy']
    print('DNN Classifier Accuracy: {0:f}'.format(accuracy_score))

Now we can call the functions that define and train our classifiers.  Let's start with the LinearClassifier, which won't be very accurate. 


In [ ]:
print("Running Linear classifier ...")
define_and_run_linear_classifier(1000, getNewPath())

Now, let's run the DNN Classifier.  First, let's try it with a .1 learning rate.

In [ ]:
print("Running DNN classifier with .1 learning rate...")
classifier = define_and_run_dnn_classifier(5000, getNewPath(), lr=.1)

Now, let's run it with a .5 learning rate.

In [ ]:
print("Running DNN classifier with .1 learning rate...")
classifier = define_and_run_dnn_classifier(5000, getNewPath(), lr=.5)

To compare your results, start up TensorBoard as follows in a new terminal window. (If you get a 'not found' error, make sure you've activated your virtual environment in that new window):

```sh
$ tensorboard --logdir=/tmp/tfmodels/mnist_tflearn
```